In [1]:
import pytorch_lightning as pl
import pytorch_lightning.callbacks as pl_callbacks
import torch
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import eq


from eq.data import Catalog, InMemoryDataset, Sequence, default_catalogs_dir

In [3]:
catalog1 = eq.catalogs.SCEDC(include_loc=False)
catalog2 = eq.catalogs.ANSS_MultiCatalog(    
    num_sequences=99,
    t_end_days=1*365,
    mag_completeness=4.5,
    minimum_mainshock_mag=6.0,
    include_loc=False,
    include_depth=False
)
# catalog3=eq.catalogs.White() 

Loading existing catalog from /home/gcl/RA/jonahk/recast/data/SCEDC.
Loading existing catalog from /home/gcl/RA/jonahk/recast/data/ANSS_MultiCatalog.


In [4]:

def combine_catalogs_sequences(seqlist):
    sequences = []
    for seq in seqlist:
        sequences.append(seq)
    return InMemoryDataset(sequences=sequences)

def build_seqlist(catalogs):
    train_sequences = []
    for catalog in catalogs:
        for seq in range(len(catalog)):
            train_sequences.append(catalog[seq])
    return train_sequences

def subtract_magnitudes(sequences, mag_completeness):
    for seq in sequences:
        seq.mag -= mag_completeness

In [5]:
catalog1.train[0].mag

tensor([2.2600, 2.3700, 2.1600,  ..., 2.3500, 2.0600, 2.1000])

In [6]:

#subtract mag_completeness from catalogs
subtract_magnitudes(catalog1.train, 2.0)
subtract_magnitudes(catalog2.train, 4.5)
subtract_magnitudes(catalog1.val, 2.0)
subtract_magnitudes(catalog2.val, 4.5)
subtract_magnitudes(catalog1.test, 2.0)
subtract_magnitudes(catalog2.test, 4.5)

#make the 2 catalogs into a list for training and val data
catalogs_train = []
catalogs_train.append(catalog1.train)
catalogs_train.append(catalog2.train)

catalogs_val = []
catalogs_val.append(catalog1.val)
catalogs_val.append(catalog2.val)

catalogs_test = []
catalogs_test.append(catalog1.test)
catalogs_test.append(catalog2.test)

#build list of catalog sequences
seqlist_train = build_seqlist(catalogs_train)
seqlist_val = build_seqlist(catalogs_val)
seqlist_test = build_seqlist(catalogs_test)

#combined the sequences into one memory i=object to train on
combined_cat_train = combine_catalogs_sequences(seqlist_train)
combined_cat_val = combine_catalogs_sequences(seqlist_val)
combined_cat_test = combine_catalogs_sequences(seqlist_test)

In [7]:
catalog1.train[0].mag

tensor([0.2600, 0.3700, 0.1600,  ..., 0.3500, 0.0600, 0.1000])

In [8]:
dl_combined_train = combined_cat_train.get_dataloader( batch_size=5, shuffle=True)
dl_combined_val = combined_cat_val.get_dataloader( batch_size=5, shuffle=True)
test = catalog1.test.get_dataloader( batch_size=1, shuffle=True)

In [9]:
T = combined_cat_train.sequences[0].t_end
N = np.mean([len(seq) for seq in combined_cat_train])
mag_mean = np.mean([combined_cat_train.sequences[0].mag.mean().item() for seq in combined_cat_train])
tau_mean = T/N
mag_completness = 0.6

anss_double_model = eq.models.RecurrentTPP(
    mag_mean = mag_mean,
    tau_mean = tau_mean,
    mag_completeness = mag_completness,
    learning_rate=1e-3,
)
    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=50, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=1000, callbacks=[checkpoint, early_stopping, progress_bar],log_every_n_steps=1)

trainer.fit(anss_double_model, dl_combined_train, dl_combined_val)
checkpoint.best_model_path
anss_double_model.load_from_checkpoint(checkpoint.best_model_path)
trainer.test(anss_double_model, test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name          ┃ Type    ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 0 │ hypernet_time │ Linear  │  3.2 K │
│ 1 │ hypernet_mag  │ Linear  │     33 │
│ 2 │ rnn           │ GRU     │  3.5 K │
│ 3 │ dropout       │ Dropout │      0 │
└───┴───────────────┴─────────┴────────┘

Trainable params: 6.7 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.7 K                                                                                                
Total estimated model params size (MB): 0

Output()

/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/
data_connector.py:490: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly 
recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(

/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/
data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be
a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus on 
this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/gcl/RA/jonahk/miniconda3/envs/DS-discovery/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/
data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may 
be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 64 which is the number of cpus 
on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(

In [ ]:
#Compare results to single catalog training
single_train = catalog1.train.get_dataloader(batch_size=1, shuffle=True)
single_val = catalog1.val.get_dataloader(batch_size=1, shuffle=True)

T = catalog1.train[0].t_end

N = np.mean([len(seq) for seq in catalog1.train])
mag_mean = np.mean([catalog1.train[0].mag.mean().item() for seq in catalog1.train])
tau_mean = T/N

single_model = eq.models.RecurrentTPP(
    mag_mean = mag_mean,
    tau_mean = tau_mean,
    mag_completeness=catalog1.metadata['mag_completeness'],
    learning_rate=1e-3,
)

    # ModelCheckpoints saves the model with the best validation loss
checkpoint = pl_callbacks.ModelCheckpoint(monitor="total_val_loss")

    # EarlyStopping stops training if the validation loss doesn't improve by more than 1e-3 for 20 epochs
early_stopping = pl_callbacks.EarlyStopping(monitor="total_val_loss", patience=10, min_delta=1e-5)

    # RichProgressBar adds a nice and more functional progress bar
progress_bar = pl_callbacks.RichProgressBar()

    # Trainer set up training and validation loops with previous specs
trainer = pl.Trainer(devices=1,max_epochs=1000, callbacks=[checkpoint, early_stopping, progress_bar],log_every_n_steps=1)

trainer.fit(single_model, single_train, single_val)
checkpoint.best_model_path
single_model.load_from_checkpoint(checkpoint.best_model_path)
trainer.test(single_model, test)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


┏━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name          ┃ Type    ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━┩
│ 0 │ hypernet_time │ Linear  │  3.2 K │
│ 1 │ hypernet_mag  │ Linear  │     33 │
│ 2 │ rnn           │ GRU     │  3.5 K │
│ 3 │ dropout       │ Dropout │      0 │
└───┴───────────────┴─────────┴────────┘

Trainable params: 6.7 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.7 K                                                                                                
Total estimated model params size (MB): 0

Output()

`Trainer.fit` stopped: `max_epochs=1000` reached.


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Output()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   Runningstage.testing    ┃                           ┃
┃          metric           ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │    -14.236465454101562    │
│      total_test_loss      │    -14.236465454101562    │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': -14.236465454101562, 'total_test_loss': -14.236465454101562}]

In [13]:
catalog1.test[0]

Sequence(
  inter_times: [125663],
  arrival_times: [125662],
  t_start: 0.0,
  t_end: 14243.998,
  t_nll_start: 12053.0,
  mag: [125662]
)

In [14]:
t_forecast = 4000
duration = 7
num_samples = 10_000

test_seq = catalog1.test[0]
# Past events that we condition on
past_seq = test_seq.get_subsequence(0, t_forecast)
# Observed events in the 7-day forecast window
observed_seq = test_seq.get_subsequence(t_forecast, t_forecast + duration)

ValueError: It should hold 0 <= t_start <= t_nll_start < t_end. Received 0.0, 12053.0, 4000.0.

In [ ]:
if torch.cuda.is_available():
    anss_double_model.cuda()
    past_seq.cuda()

In [ ]:
multi_forecast = anss_double_model.sample(batch_size=num_samples, duration=duration, past_seq=past_seq, return_sequences=True)
single_forecast = single_model.sample(batch_size=num_samples, duration=duration, past_seq=past_seq, return_sequences=True)

In [ ]:
eq.visualization.visualize_trajectories(test_seq, multi_forecast)

In [ ]:
eq.visualization.visualize_trajectories(test_seq, single_forecast)